欢迎来到本周第一个作业。你将建立一个将人类可读日期（“2009年6月25日”）转换为机器可读日期（“2009-06-25”）的神经机器翻译（NMT）模型。 你将使用注意力机制来执行此操作，这是模型序列中最尖端的一个序列。

In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


# 将人类可读日期翻译为机器可读日期
你将创建的模型可用于从一种语言翻译为另一种语言，如从英语翻译为印地安语。 但是，语言翻译需要大量的数据集，并且通常需要几天的GPU训练。 在不使用海量数据的情况下，为了让你有机会尝试使用这些模型，我们使用更简单的“日期转换”任务。

网络以各种可能格式（例如“1958年8月29日”，“03/30/1968”，“1987年6月24日”）写成的日期作为输入，并将它们转换成标准化的机器可读的日期（例如“1958 -08-29“，”1968-03-30“，”1987-06-24“），让网络学习以通用机器可读格式YYYY-MM-DD输出日期。

## 数据集
我们将在一个包含10000个可读日期的数据集及其等效的标准化机器可读日期上训练模型。 执行以下命令加载数据并查看一些示例。

In [5]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)
dataset[:10]

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 14542.82it/s]


[('16 june 2009', '2009-06-16'),
 ('monday june 29 2015', '2015-06-29'),
 ('saturday february 13 1999', '1999-02-13'),
 ('1/27/91', '1991-01-27'),
 ('thursday april 20 1972', '1972-04-20'),
 ('friday december 3 1971', '1971-12-03'),
 ('26 mar 2015', '2015-03-26'),
 ('9 06 95', '1995-06-09'),
 ('monday march 13 1989', '1989-03-13'),
 ('saturday november 14 1970', '1970-11-14')]

其中
- dataset：一个二元组列表（人类可读日期，机器可读日期）
- human_vocab: 人类可读日期中使用到的所有字符到字典索引值的映射
- machine_vocab: 机器可读日期中使用到的所有字符到字典索引值的映射(与human_vocab的索引没必要完全一致)
- inv_machine_vocab: machine_vocab 的翻转映射，从索引映射到字符

预处理数据并将原始文本数据映射到索引。 我们还将使用Tx=30（假设它是人类可读日期的最大长度;如果得到更长的输入，将不得不截断它）并且Ty=10（因为“YYYY-MM-DD”是10个字符）

In [9]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


目前已有：

- X: 经过处理的训练集中人类可读日期，其中每个字符都替换为其在human_vocab中映射到的索引。 每个日期用特殊字符进一步填充为Tx长度。 X.shape =（m，Tx）
- Y: 经过处理的训练集中机器可读日期，其中每个字符都替换为其在machine_vocab中映射到的索引。 你应该有Y.shape =（m，Ty）。
- Xoh：X的one-hot向量，Xoh.shape = (m，Tx，len(human_vocab))
- Yoh：Y的one-hot向量，Yoh.shape = (m，Tx，len(machine_vocab))。

In [14]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 16 june 2009
Target date: 2009-06-16

Source after preprocessing (indices): [ 4  9  0 22 31 25 17  0  5  3  3 12 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 3  1  1 10  0  1  7  0  2  7]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


# 采用注意力机制的机器翻译
如果你需要将一段法文翻译成英文，你不会阅读整段然后关闭书本再翻译。 即使在翻译过程中，您也会阅读/重读，并专注于你正在翻译的部分。

注意力机制告诉神经翻译模型什么时候应该关注什么
## 注意力机制
在这一部分，你将会实现注意力机制，图中展示了注意力机制的工作原理。上图是注意力机制的模型，下图是每步注意力变量$\alpha^{\langle t, t' \rangle}$的计算,该变量将用于计算每步输出的$context^{\langle t \rangle}$, 其中 ($t=1, \ldots, T_y$). 

<table>
    <tr>
        <td> 
            <img src="images/attn_model.png" style="width:500;height:500px;"><br>
        </td> 
        <td> 
            <img src="images/attn_mechanism.png" style="width:500;height:500px;"><br>
        </td>
    </tr>
</table>
<caption><center><b>Figure 1</b>: Neural machine translation with attention</center></caption>
下面是关于模型的一些属性说明：

- 模型中有两个单独的LSTM。 
  - 图片底部的是双向LSTM，处在关注机制之前，将其称为pre-attention Bi-LSTM。
  - 图片顶部的LSTM出现在关注机制之后，将其称为post-attention LSTM。
  - pre-attention Bi-LSTM经过$T_x$个时间步。
  - post-attention LSTM经历$T_y$个时间步。
- post-attention LSTM将$s^{\langle t \rangle}, c^{\langle t \rangle}$从一个时间步传给下一个时间步。 
  - 在视频课程中，我们仅使用了基本的RNN作为post-attention 序列模型，因此RNN输出激活状态为$s^{\langle t\rangle}$。
  - 这里我们使用LSTM，因此LSTM具有输出激活状态$s^{\langle t\rangle}$和隐藏单元状态$c^{\langle t\rangle}$。
  - 然而，与先前的文本生成示例（例如第1周的恐龙）不同，在该模型中，在时间t的post-activation LSTM不会将具体生成的$y^{\langle t-1 \rangle}$ 作为输入; 而只是$s^{\langle t\rangle}$和$c^{\langle t\rangle}$作为输入。
  - 我们以这种方式设计了模型，因为（与邻近字符高度相关的语言生成不同），在YYYY-MM-DD日期中，前一个字符与下一个字符之间的依赖性不强。
- 我们使用$a^{\langle t \rangle} = [\overrightarrow{a}^{\langle t \rangle}; \overleftarrow{a}^{\langle t \rangle}]$ 来表示pre-attention Bi-LSTM的前向激活函数和反向激活函数。
- 使用RepeatVector节点复制$T_x$次$s^{\langle t-1 \rangle}$的值，然后连接$s^{\langle t-1 \rangle}$和$a^{\langle t \rangle}$ 来计算$e^{\langle t, t' \rangle}$>, 然后$e^{\langle t, t' \rangle}$通过softmax来计算 $\alpha^{\langle t, t' \rangle}$。
我们将在下面解释如何在Keras中使用RepeatVector和Concatenation。

 下面实现模型，首先需要实现one_step_attention() 和 model():
- 1.one_step_attention(): 在第t个时间步，利用Bi-LSTM的所有隐藏状态 ($[a^{\langle 1 \rangle},a^{\langle 2 \rangle}, ..., a^{\langle T_x \rangle}]$)  和第二个LSTM的之前的隐藏状态($s^{\langle t-1 \rangle}$)，函数 one_step_attention() 可以计算出注意力权重($[\alpha^{\langle t,1 \rangle},\alpha^{\langle t,2 \rangle}, ..., \alpha^{\langle t,T_x \rangle}]$)然后输出上下文向量(见图二) 
$$context^{\langle t \rangle} = \sum_{t' = 0}^{T_x} \alpha^{\langle t,t' \rangle}a^{\langle t' \rangle}\tag{1}$$ 
- 2.model(): 实现了完整的模型。首先执行Bi-LSTM得到 $[a^{\langle 1 \rangle},a^{\langle 2 \rangle}, ..., a^{\langle T_x \rangle}]$， 
然后在for循环中执行$T_y$次one_step_attention()。 
每次迭代都是利用$c^{\langle t\rangle}$计算下一个LSTM 并通过softmax层产生预测值$\hat{y}^{\langle t \rangle}$。

**练习：实现 one_step_attention()**

model() 将会在for循环中执行$T_y$次one_step_attention()，重要的是TyTy的每个副本都具有相同的权重，换言之，$T_y$的所有时间步共享权重。

下面是如何在Keras中利用共享权重进行实现层 
1. 定义层对象(例如全局变量) 
2. 前向传播时调用这些对象

In [17]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

现在可以用这些层实现 one_step_attention() 了。为了在各层传播Keras 的对象X，调用layer(X) (或者layer([X, Y])如果需要多个输入的话)，densor(X) 将通过上面定义的Denser(1)层进行传播。

In [18]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###
    
    return context

**练习：实现 model()**

In [19]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

现在你可以在for循环中使用Ty次这些层来生成输出了，它们的参数将不会重新初始化。

1. 将输入传给一个 Bidirectional LSTM
2. 循环 $t = 0, \dots, T_y-1$: 
   1. 在$[\alpha^{<t,1>},\alpha^{<t,2>}, ..., \alpha^{<t,T_x>}]$  和$s^{<t-1>}$上执行 one_step_attention() 得到上下文向量$context^{<t>}$
   2. 将$context^{<t>}$输入post-attention LSTM。记得使用`initial_state= [previous hidden state, previous cell state]`传入前面的隐藏状态$s^{<t-1>}$和LSTM的单元状态$c^{<t-1>}$。得到新的隐藏状态$s^{<t>}$和新的单元状态$c^{<t>}$
   3. 将$s^{<t>}$传给softmax层，得到输出
   4. 将输出添加到输出列表中进行保存
3. 创建你的Keras模型实例，模型有三个输入（“inputs”, $s^{<0>}$ and $c^{<0>}$），输出一个列表”output”

In [20]:
# GRADED FUNCTION: model

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True), input_shape=(m, Tx, n_a * 2))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs=[X, s0, c0], outputs=outputs)
    
    ### END CODE HERE ###
    
    return model



In [21]:
#创建模型
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

和往常一样，在Keras中创建模型之后，需要编译模型。定义损失函数，优化器和指标。使用 categorical_crossentropy 损失，Adam优化器(learning rate = 0.005, β1=0.9, β2=0.999, decay = 0.01) 和 [‘accuracy’]指标来编译模型。

In [23]:
### START CODE HERE ### (≈2 lines)
opt = Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss = 'categorical_crossentropy',optimizer=opt, metrics = ['accuracy'])
### END CODE HERE ###

最后一步是定义模型的输入和输出：

- 你已经有了包含所有训练样本的(m=10000,Tx=30)的X
- 需要创建 s0,c0 来为 post_activation_LSTM_cell 进行0值初始化
- model() 的输出为11个(m, Ty)元素的列表。所以：outputs[i][0], …, outputs[i][Ty] 表示第i的样本(X[i])的标签字符。更一般的说，outputs[i][j] 表示第i个训练样本的第j个字母的标签字符。

In [24]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
outputs = list(Yoh.swapaxes(0,1))

In [26]:
model.fit([Xoh, s0, c0], outputs, epochs=100, batch_size=100)

Epoch 1/100


 2700/10000 [=======>......................] - ETA: 15s - loss: 21.0406 - dense_3_loss: 2.9821 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6400 - dense_3_acc_2: 0.2200 - dense_3_acc_3: 0.0000e+00 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.0000e+ - ETA: 15s - loss: 20.8734 - dense_3_loss: 2.8940 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6200 - dense_3_acc_2: 0.2200 - dense_3_acc_3: 0.0000e+00 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.0000e+ - ETA: 15s - loss: 20.8461 - dense_3_loss: 2.8527 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6400 - dense_3_acc_2: 0.1933 - dense_3_acc_3: 0.0000e+00 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.0000e+ - ETA: 15s - loss: 20.861

 8200/10000 [=======================>......] - ETA: 7s - loss: 20.3653 - dense_3_loss: 2.8667 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6089 - dense_3_acc_2: 0.2227 - dense_3_acc_3: 0.0064 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.009 - ETA: 7s - loss: 20.3542 - dense_3_loss: 2.8665 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6093 - dense_3_acc_2: 0.2221 - dense_3_acc_3: 0.0080 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.010 - ETA: 7s - loss: 20.3392 - dense_3_loss: 2.8650 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6100 - dense_3_acc_2: 0.2226 - dense_3_acc_3: 0.0100 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.011 - ETA: 6s - loss: 20.3277 - dense_3_loss: 2.8650 

10000/10000 [==============================] - ETA: 2s - loss: 20.0701 - dense_3_loss: 2.8295 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6146 - dense_3_acc_2: 0.2213 - dense_3_acc_3: 0.0346 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0111 - dense_3_acc_9: 0.041 - ETA: 2s - loss: 20.0603 - dense_3_loss: 2.8273 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6144 - dense_3_acc_2: 0.2218 - dense_3_acc_3: 0.0354 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0124 - dense_3_acc_9: 0.042 - ETA: 2s - loss: 20.0510 - dense_3_loss: 2.8272 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.6141 - dense_3_acc_2: 0.2226 - dense_3_acc_3: 0.0358 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.0132 - dense_3_acc_9: 0.042 - ETA: 2s - loss: 20.0415 - dense_3_loss: 2.8243 - dense_3_ac

 2800/10000 [=======>......................] - ETA: 16s - loss: 19.2860 - dense_3_loss: 2.7698 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.5300 - dense_3_acc_2: 0.1400 - dense_3_acc_3: 0.0500 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.1700 - dense_3_acc_9: 0.07 - ETA: 17s - loss: 19.1891 - dense_3_loss: 2.7133 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.5800 - dense_3_acc_2: 0.1550 - dense_3_acc_3: 0.0450 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.1250 - dense_3_acc_9: 0.10 - ETA: 16s - loss: 19.0569 - dense_3_loss: 2.6708 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.5933 - dense_3_acc_2: 0.1733 - dense_3_acc_3: 0.0367 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.1100 - dense_3_acc_9: 0.11 - ETA: 16s - loss: 18.9934 - dense_3_loss: 2.6647 - dense_3_a

 8400/10000 [========================>.....] - ETA: 6s - loss: 18.6458 - dense_3_loss: 2.6803 - dense_3_acc: 0.5454 - dense_3_acc_1: 0.6067 - dense_3_acc_2: 0.2540 - dense_3_acc_3: 0.0679 - dense_3_acc_4: 0.9998 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.2084 - dense_3_acc_9: 0.118 - ETA: 6s - loss: 18.6423 - dense_3_loss: 2.6817 - dense_3_acc: 0.5462 - dense_3_acc_1: 0.6064 - dense_3_acc_2: 0.2547 - dense_3_acc_3: 0.0676 - dense_3_acc_4: 0.9998 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.2090 - dense_3_acc_9: 0.118 - ETA: 6s - loss: 18.6317 - dense_3_loss: 2.6818 - dense_3_acc: 0.5492 - dense_3_acc_1: 0.6083 - dense_3_acc_2: 0.2551 - dense_3_acc_3: 0.0673 - dense_3_acc_4: 0.9998 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0000e+00 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 0.2103 - dense_3_acc_9: 0.119 - ETA: 6s - loss: 18.6265 - dense_3_loss: 2.6811 - dense_3_acc: 0.5502 - 

KeyboardInterrupt: 

In [27]:
model.load_weights('models/model.h5')

In [28]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
    source = np.expand_dims(source, axis=0)
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

source: 3 May 1979
output: 1979-05-03
source: 5 April 09
output: 2009-05-05
source: 21th of August 2016
output: 2016-08-21
source: Tue 10 Jul 2007
output: 2007-07-10
source: Saturday May 9 2018
output: 2018-05-09
source: March 3 2001
output: 2001-03-03
source: March 3rd 2001
output: 2001-03-03
source: 1 March 2001
output: 2001-03-01
